In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms
from collections import Counter
from torchmetrics import AUROC
from statistics import mean, stdev

In [2]:
seed = 123
torch.manual_seed(seed)

In [3]:
transforms = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05, brightness = .05),
    torchvision.transforms.RandomHorizontalFlip(p = 0.5),
    torchvision.transforms.RandomVerticalFlip(p = 0.5),
    torchvision.transforms.RandomRotation(degrees = (0, 180)),
    ])

In [4]:
train_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/train', transform = transforms )
test_data = torchvision.datasets.ImageFolder(root = '/mnt/d/MHIST/test', transform = transforms)

In [5]:
train_size = int(0.9 * len(train_data))
val_size = len(train_data) - train_size

In [6]:
train_dataset, val_dataset = torch.utils.data.random_split(train_data, [train_size, val_size])

In [7]:
train_dataset.dataset.class_to_idx
print(dict(Counter(train_dataset.dataset.targets)))

{0: 1545, 1: 630}


In [8]:
batch_size = 32

In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size= batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= batch_size)

In [10]:
model = torchvision.models.resnet18(pretrained = True)
model.fc = nn.Sequential(nn.Linear(512,2))
model = model.to(device = 'cuda')

In [11]:
loss = torch.nn.CrossEntropyLoss()

In [12]:
def train(num_epochs):
    best_accuracy = 0.0
    
    print("Starting training...")
    
    for epoch in range(1, num_epochs + 1):
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_val_loss = 0.0 
        total = 0.0        
        # Training loop
        
        train_auroc = AUROC(pos_label = 1, num_classes = 2)
        val_auroc = AUROC(pos_label = 1, num_classes = 2)
        
        for data in train_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            optimizer.zero_grad()             
            predicted_outputs = model(inputs)   
            train_loss = loss(predicted_outputs, outputs)   
            train_loss.backward()   
            optimizer.step()        
            running_train_loss +=train_loss.item()
            train_auroc_batch = train_auroc(predicted_outputs, outputs)
                        
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(train_loader) 
        avg_train_auroc = train_auroc.compute()
        
        # Validation Loop 
        with torch.no_grad(): 
            model.eval()
            for data in val_loader:
                inputs, outputs = data
                inputs = inputs.to(device = 'cuda')
                outputs = outputs.to(device = 'cuda')
                predicted_outputs = model(inputs)
                val_loss = loss(predicted_outputs, outputs)
                
                _, predicted = torch.max(predicted_outputs, 1)
                running_val_loss += val_loss.item()
                total += outputs.size(0)
                running_accuracy += (predicted == outputs).sum().item()
                val_auroc_batch = val_auroc(predicted_outputs, outputs)
         
                
        # Calculate validation loss value 
        val_loss_value = running_val_loss/len(val_loader) 
        avg_val_auroc = val_auroc.compute()        
        # Calculate accuracy as the number of correct pred
        accuracy = (100 * running_accuracy / total)     
        
        print('EPOCH', epoch,
              'Training Loss: %.4f' %train_loss_value, 
              'Train AUROC: %.4f' %avg_train_auroc,
              'Validation Loss: %.4f' %val_loss_value,
              'Validation AUROC: %.4f' %avg_val_auroc,
              'Accuracy %d %%' % (accuracy))
        train_auroc.reset()
        val_auroc.reset()

In [13]:
def test(model):
    test_auroc = AUROC(pos_label = 1, num_classes = 2)
    with torch.no_grad():
        model.eval()
        for data in test_loader:
            inputs, outputs = data
            inputs = inputs.to(device = 'cuda')
            outputs = outputs.to(device = 'cuda')
            predicted_outputs = model(inputs)
            _, predicted = torch.max(predicted_outputs, 1)
            test_auroc_batch = test_auroc(predicted_outputs, outputs)
    
    avg_test_auroc = test_auroc.compute()
    return avg_test_auroc

In [14]:
results = []

for i in range(9):
    
    seed = i
    torch.manual_seed(seed)
    
    model = torchvision.models.resnet18(pretrained = True)
    model.fc = nn.Sequential(nn.Linear(512,2))
    model = model.to(device = 'cuda')
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 0.01)
    train(30)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 0.01)
    train(10)
    
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001, momentum = 0.7, weight_decay = 0.01)
    train(10)
    
    results.append(test(model))

Starting training...


/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Argument `pos_label` should be `None` when running multiclass precision recall curve. Got 1
  warnings.warn(*args, **kwargs)


EPOCH 1 Training Loss: 0.5185 Train AUROC: 0.7130 Validation Loss: 0.4748 Validation AUROC: 0.8393 Accuracy 78 %
EPOCH 2 Training Loss: 0.4982 Train AUROC: 0.7674 Validation Loss: 0.6480 Validation AUROC: 0.8369 Accuracy 61 %
EPOCH 3 Training Loss: 0.4838 Train AUROC: 0.7767 Validation Loss: 0.3875 Validation AUROC: 0.8703 Accuracy 84 %
EPOCH 4 Training Loss: 0.4317 Train AUROC: 0.8345 Validation Loss: 0.3583 Validation AUROC: 0.8918 Accuracy 83 %
EPOCH 5 Training Loss: 0.4313 Train AUROC: 0.8362 Validation Loss: 0.3666 Validation AUROC: 0.8956 Accuracy 83 %
EPOCH 6 Training Loss: 0.3944 Train AUROC: 0.8688 Validation Loss: 0.3314 Validation AUROC: 0.9152 Accuracy 85 %
EPOCH 7 Training Loss: 0.4410 Train AUROC: 0.8263 Validation Loss: 0.3694 Validation AUROC: 0.8994 Accuracy 85 %
EPOCH 8 Training Loss: 0.4053 Train AUROC: 0.8631 Validation Loss: 0.3748 Validation AUROC: 0.9205 Accuracy 83 %
EPOCH 9 Training Loss: 0.3440 Train AUROC: 0.9022 Validation Loss: 0.2800 Validation AUROC: 0.93

/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/home/alon/anaconda3/envs/ml/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


EPOCH 12 Training Loss: 0.2890 Train AUROC: 0.9305 Validation Loss: 0.2825 Validation AUROC: 0.9434 Accuracy 89 %
EPOCH 13 Training Loss: 0.3184 Train AUROC: 0.9171 Validation Loss: 0.2917 Validation AUROC: 0.9401 Accuracy 88 %
EPOCH 14 Training Loss: 0.2974 Train AUROC: 0.9278 Validation Loss: 0.3349 Validation AUROC: 0.9228 Accuracy 85 %
EPOCH 15 Training Loss: 0.2983 Train AUROC: 0.9295 Validation Loss: 0.2723 Validation AUROC: 0.9521 Accuracy 88 %
EPOCH 16 Training Loss: 0.3035 Train AUROC: 0.9282 Validation Loss: 0.4028 Validation AUROC: 0.9499 Accuracy 83 %
EPOCH 17 Training Loss: 0.2748 Train AUROC: 0.9391 Validation Loss: 0.2393 Validation AUROC: 0.9558 Accuracy 89 %
EPOCH 18 Training Loss: 0.2755 Train AUROC: 0.9427 Validation Loss: 0.6201 Validation AUROC: 0.9309 Accuracy 77 %
EPOCH 19 Training Loss: 0.3623 Train AUROC: 0.8877 Validation Loss: 0.2628 Validation AUROC: 0.9489 Accuracy 88 %
EPOCH 20 Training Loss: 0.2862 Train AUROC: 0.9357 Validation Loss: 0.2692 Validation AU

In [15]:
results

[tensor(0.9346, device='cuda:0'),
 tensor(0.9409, device='cuda:0'),
 tensor(0.9382, device='cuda:0'),
 tensor(0.9395, device='cuda:0'),
 tensor(0.9370, device='cuda:0'),
 tensor(0.9357, device='cuda:0'),
 tensor(0.9420, device='cuda:0'),
 tensor(0.9362, device='cuda:0'),
 tensor(0.9381, device='cuda:0')]

In [34]:
results = [result.item() for result in results] 

In [55]:
print('Mean:',"{:.3f}".format(mean(results)*100),'\nStandard deviation:', "{:.3f}".format(stdev(results)*100))

Mean: 93.801 
Standard deviation: 0.245
